In [1]:
import base64
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate
import os
import re

In [2]:
etc_pmt = """
    你是一位先进的AI，能够处理并从图像中提取文本。图像中包含用黑色文本表示的问题和用红色文本表示的答案。你的任务是提取问题和答案，并按指定格式输出。

    说明：
    1. 从提供的图像中提取所有文本。
    2. 将问题（黑色文本）与答案（红色文本）分开。
    3. 按以下格式输出提取的内容：
        ### 题目:
        [问题文本]

        ### 答案:
        [答案文本]
"""

In [6]:
path = "./test_set/"
dir = os.listdir(path)
record = []
log = []
problem_set = []
for img in dir[:]:
    if '.png' not in img: continue
    print(f"----processing {img}----")
    cur = {}
    with open(path+img, 'rb') as file:
        image_data = base64.b64encode(file.read()).decode("utf-8")
    llm = ChatOpenAI(model='gpt-4o', max_tokens=1000)
    message = HumanMessage(
        [
            {"type": "text", "text": etc_pmt},
            {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{image_data}"}},
        ],
    )
    try:
        raw_output = llm.invoke([message])
    except Exception as e:
        log.append(img+e)
        continue
    record.append(raw_output)
    # 提取问题的正则表达式
    question_pattern = re.compile(r"### 题目:\n(.*?)\n\n### 答案:", re.S)

    # 提取答案的正则表达式
    answer_pattern = re.compile(r"### 答案:\n(.*)", re.S)

    question_match = question_pattern.search(raw_output.content)
    if question_match:
        question = question_match.group(1).strip()
        cur['problem'] = question
    
    answer_match = answer_pattern.search(raw_output.content)
    if answer_match:
        answer = answer_match.group(1).strip()
        cur['answer'] = answer
    problem_set.append(cur)

----processing slide_1817.png----
----processing slide_2265.png----
----processing slide_2259.png----
----processing slide_1988.png----
----processing slide_1989.png----
----processing slide_2258.png----
----processing slide_2264.png----
----processing slide_2099.png----
----processing slide_3387.png----
----processing slide_1816.png----
----processing slide_2104.png----
----processing slide_1948.png----
----processing slide_2272.png----
----processing slide_2267.png----
----processing slide_1949.png----
----processing slide_2059.png----
----processing slide_1815.png----
----processing slide_2101.png----
----processing slide_1971.png----
----processing slide_1970.png----
----processing slide_2060.png----
----processing slide_1973.png----
----processing slide_2414.png----
----processing slide_1121.png----
----processing slide_1135.png----
----processing slide_1900.png----
----processing slide_1928.png----
----processing slide_2367.png----
----processing slide_2415.png----
----processing

In [7]:
import json
with open("temp.json", "w", encoding='utf-8') as f:
    json.dump(problem_set, f, ensure_ascii=False, indent=4)

In [8]:
len(problem_set)

100

In [18]:
print(record[3].content[4:-4])

### 题目:
在甲处劳动的工人有35人，在乙处劳动的工人有19人。现在另调21人去支援，使得在甲处的人数为乙处人数的3/2倍，应调往甲、乙两处各多少人？

### 答案:
解：设调往甲处x人，调往乙处y人。
依题意，得
x + y = 21，
35 + x = 3/2 (19 + y)，
解得
x = 10, 
y = 11。

答：应调往甲处10人，调往乙处11人。


In [19]:
import re

text = """
### 题目:
在甲处劳动的工人有35人，在乙处劳动的工人有19人。现在另调21人去支援，使得在甲处的人数为乙处人数的3/2倍，应调往甲、乙两处各多少人？

### 答案:
解：设调往甲处x人，调往乙处y人。
依题意，得
x + y = 21，
35 + x = 3/2 (19 + y)，
解得
x = 10, 
y = 11。

答：应调往甲处10人，调往乙处11人。
"""

# 提取问题的正则表达式
question_pattern = re.compile(r"### 题目:\n(.*?)\n\n### 答案:", re.S)

# 提取答案的正则表达式
answer_pattern = re.compile(r"### 答案:\n(.*)", re.S)

# 提取问题
question_match = question_pattern.search(text)
if question_match:
    question = question_match.group(1).strip()
    print("问题:")
    print(question)

# 提取答案
answer_match = answer_pattern.search(text)
if answer_match:
    answer = answer_match.group(1).strip()
    print("答案:")
    print(answer)


问题:
在甲处劳动的工人有35人，在乙处劳动的工人有19人。现在另调21人去支援，使得在甲处的人数为乙处人数的3/2倍，应调往甲、乙两处各多少人？
答案:
解：设调往甲处x人，调往乙处y人。
依题意，得
x + y = 21，
35 + x = 3/2 (19 + y)，
解得
x = 10, 
y = 11。

答：应调往甲处10人，调往乙处11人。
